In [ ]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets
import random

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# train_data, test_data = datasets.TREC.splits(TEXT, LABEL, fine_grained=False)
# train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [ ]:
TEXT = data.Field(tokenize = 'spacy', tokenizer_language = 'en_core_web_sm')
# EVENT_DATE = data.Field()
# SOURCE = data.Field()
# FATALITIES = data.Field()
# EVENT_TYPE = data.LabelField()
# SUB_EVENT_TYPE = data.LabelField()
# ACTOR1 = data.LabelField()
# INTER1 = data.LabelField()
# ACTOR2 = data.LabelField()
# INTER2 = data.LabelField()
# INTERACTION = data.Field()
# LOCATION = data.Field()

LABEL = data.LabelField()

# create tuples representing the columns
fields = [
          ('text', TEXT),
          (None, None),
          (None, None),
          (None, None),
          (None, None),
          (None, None),
          (None, None),
          (None, None),
          (None, None),
          ('label', LABEL),
          (None, None),
          (None, None),
]

# load the dataset in json format
# https://dzlab.github.io/dltips/en/pytorch/torchtext-datasets/
train_data, valid_data, test_data = data.TabularDataset.splits(
   path = '',
   train = 'task_1_information_extraction_train_20k.csv',
   validation = 'task_1_information_extraction_valid_5k.csv',
   test = 'task_1_information_extraction_valid_5k.csv',
   format = 'csv',
   fields = fields,
   skip_header = True
)

# check an example
print(vars(train_data[0]))

# train_data, valid_data = train_data.split(random_state = random.seed(SEED))

{'text': ['Three', 'people', 'were', 'killed', 'while', '27', 'others', 'injured', 'when', 'a', 'Peshawar', '-', 'bound', 'train', 'hit', 'a', 'bomb', 'planted', 'by', 'unidentified', 'militants', 'on', 'railway', 'tracks', 'in', 'Tul', 'town', 'in', 'Jacobabad', 'district', 'in', 'Sindh', '.'], 'label': '7'}


Let's look at one of the examples in the training set.

In [ ]:
vars(train_data[0])

{'label': '7',
 'text': ['Three',
  'people',
  'were',
  'killed',
  'while',
  '27',
  'others',
  'injured',
  'when',
  'a',
  'Peshawar',
  '-',
  'bound',
  'train',
  'hit',
  'a',
  'bomb',
  'planted',
  'by',
  'unidentified',
  'militants',
  'on',
  'railway',
  'tracks',
  'in',
  'Tul',
  'town',
  'in',
  'Jacobabad',
  'district',
  'in',
  'Sindh',
  '.']}

In [ ]:
vars(valid_data[-1])

{'label': '7',
 'text': ['Rebel',
  'fighters',
  'raped',
  '9',
  'girls',
  'at',
  'a',
  'church',
  'in',
  'Malakal',
  '[',
  'women',
  'targeted',
  ':',
  'girls',
  ']']}

Next, we'll build the vocabulary. 

In [ ]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

# LABEL.build_vocab(train_data)
LABEL.build_vocab(valid_data)

Next, we can check the labels.

The 6 labels (for the non-fine-grained case) correspond to the 6 types of Events, but can be used for any column

In [ ]:
print(LABEL.vocab.stoi)

defaultdict(None, {'7': 0, '2': 1, '1': 2, '3': 3, '0': 4, '8': 5, '4': 6, '5': 7, '6': 8})


As always, we set up the iterators.

In [ ]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    # sort_key = lambda x: len(x),
    # sort_within_batch = False,
    sort = False,
    device = device)

We'll be using the CNN model.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [sent len, batch size]
        
        text = text.permute(1, 0)
                
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [2,3,4]
OUTPUT_DIM = len(LABEL.vocab)
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

Checking the number of parameters

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,593,209 trainable parameters


Next, we'll load our pre-trained embeddings.

In [ ]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.3398,  0.2094,  0.4635,  ..., -0.2339,  0.4730, -0.0288],
        ...,
        [ 0.1519,  0.9318,  1.4014,  ...,  0.8167, -0.6117,  1.5515],
        [-0.2769,  1.9057,  0.9455,  ...,  0.0970,  1.1845,  0.7264],
        [-1.5125,  0.5574, -0.4781,  ...,  1.3716, -1.6613,  1.2968]])

Then zero the initial weights of the unknown and padding tokens.

In [ ]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

`CrossEntropyLoss` performs a *softmax* function over our model outputs and the loss is given by the *cross entropy* between that and the label. It is used when our examples exclusively belong to one of $C$ classes


In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    top_pred = preds.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

The training loop is similar to before, without the need to `squeeze` the model predictions as `CrossEntropyLoss` expects the input to be **[batch size, n classes]** and the label to be **[batch size]**.

The label needs to be a `LongTensor`, which it is by default as we did not set the `dtype` to a `FloatTensor` as before.

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text)
        
        loss = criterion(predictions, batch.label)
        
        acc = categorical_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

The evaluation loop.

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text)
            
            loss = criterion(predictions, batch.label)
            
            acc = categorical_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Next, we train our model.

In [ ]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut5-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 1m 18s
	Train Loss: 1.302 | Train Acc: 55.50%
	 Val. Loss: 1.086 |  Val. Acc: 63.47%
Epoch: 02 | Epoch Time: 1m 16s
	Train Loss: 0.896 | Train Acc: 69.57%
	 Val. Loss: 0.937 |  Val. Acc: 67.48%
Epoch: 03 | Epoch Time: 1m 16s
	Train Loss: 0.757 | Train Acc: 73.81%
	 Val. Loss: 0.882 |  Val. Acc: 70.29%
Epoch: 04 | Epoch Time: 1m 16s
	Train Loss: 0.656 | Train Acc: 77.63%
	 Val. Loss: 0.864 |  Val. Acc: 70.75%
Epoch: 05 | Epoch Time: 1m 16s
	Train Loss: 0.575 | Train Acc: 80.35%
	 Val. Loss: 0.864 |  Val. Acc: 70.91%


Finally, let's run our model on the test set!

In [ ]:
model.load_state_dict(torch.load('tut5-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.864 | Test Acc: 70.75%


We can now make a function that will predict the class of question given.

We use the `argmax` to get the highest predicted class index. We then use this index with the label vocab to get the human readable label.

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

def predict_class(model, sentence, min_len = 4):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    preds = model(tensor)
    max_preds = preds.argmax(dim = 1)
    return max_preds.item()

Now, let's try it out on a few different event notes...

In [ ]:
pred_class = predict_class(model, "A displaced woman was shot and killed by an unknown gunman after leaving the UN camp in Malakal town in South Sudan's Eastern Nile State")
print(f'Predicted class is: {pred_class} = {LABEL.vocab.itos[pred_class]}')

Predicted class is: 0 = 7


In [ ]:
pred_class = predict_class(model, "UNMISS went on patrol after hearing shooting nearby. Two women were injured and taken to the hospital. Since resumed fighting on 18/2/2014, Malakal was looted.")
print(f'Predicted class is: {pred_class} = {LABEL.vocab.itos[pred_class]}')

Predicted class is: 2 = 1


In [ ]:
pred_class = predict_class(model, "White Army fighters shot and killed a man in Malakal when he refused to hand over his money and mobile phone")
print(f'Predicted class is: {pred_class} = {LABEL.vocab.itos[pred_class]}')

Predicted class is: 0 = 7


In [ ]:
pred_class = predict_class(model, "Military forces battle with an armed group believed to be a faction of SPLM-DC killing 7. The group are blamed for the killing of a Shilluk chief earlier.")
print(f'Predicted class is: {pred_class} = {LABEL.vocab.itos[pred_class]}')

Predicted class is: 1 = 2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from sklearn.metrics import classification_report

y_pred = model.predict(x_test, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_test, y_pred_bool))